In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
from rich.progress import track
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

sys.path.append("../src")
from utils.feature import FeatureStore
from utils.common import load_pickle, dump_pickle, save_cache

In [2]:
from dataset import IndoorDataset, IndoorDataModule
from submission import IndoorTestDataset

In [20]:
d1 = torch.arange(1., 11., requires_grad=True).view(-1, 2)
d2 = torch.arange(10., 0., -1).view(-1, 2)

In [22]:
d1, d2

(tensor([[ 1.,  2.],
         [ 3.,  4.],
         [ 5.,  6.],
         [ 7.,  8.],
         [ 9., 10.]], grad_fn=<ViewBackward>),
 tensor([[10.,  9.],
         [ 8.,  7.],
         [ 6.,  5.],
         [ 4.,  3.],
         [ 2.,  1.]]))

In [24]:
loss_fn = nn.MSELoss()
loss_fn(d1, d2)

tensor(33., grad_fn=<MseLossBackward>)

In [26]:
diff = torch.abs(d1 - d2)
diff

tensor([[9., 7.],
        [5., 3.],
        [1., 1.],
        [3., 5.],
        [7., 9.]], grad_fn=<AbsBackward>)

In [28]:
tmp = torch.sum(diff, dim=1)
tmp

tensor([16.,  8.,  2.,  8., 16.], grad_fn=<SumBackward1>)

In [29]:
torch.mean(tmp)

tensor(10., grad_fn=<MeanBackward0>)